### Control Digital en Sistemas Embebidos - MSE - PRACTICA 4

In [ ]:
!pipenv install slycot
!pipenv install control
!pipenv install matplotlib

In [1]:
import numpy as np
import control as cnt
from scipy.linalg import expm
import matplotlib.pyplot as plt

Dada la siguiente función transferencia de una planta continua:

$H_{s} = (0.2s + 1) / (0.1s^2 + 0.4s + 1)$

**1)** Discretizar la planta.

In [ ]:
h = 0.1

num = np.array([0.2, 1])
den = np.array([0.1, 0.4, 1])
hs = cnt.tf(num, den)
hz = cnt.c2d(hs, h, 'zoh')
hz

In [ ]:
ss = cnt.tf2ss(hz)
ss

**2)** Obtener la respuesta al escalón en tiempo continuo y discreto.

In [ ]:
t = np.linspace(h, h * 100, 100)
u = np.concatenate((np.zeros(int(len(t) / 2)), np.ones(int(len(t) / 2))), axis=None)

In [ ]:
t_hz_open, y_hz_open = cnt.forced_response(hz, t, u)

In [ ]:
p_1 = plt.plot(y_hz_open)
plt.ylabel('u(t)')
plt.title('Respuesta al Escalon')
plt.legend('y(t)');

**3)** Aplicar un control diseñado por el método de Pole Placement de forma que el sistema ubique ambos polos en (0,5 +- j 0.3).

Primero probamos sin obtener K0

In [ ]:
poles = [0.5 + 0.3j, 0.5 - 0.3j]

K = cnt.place(ss.A, ss.B, poles)

Phi_LC = ss.A - ss.B * K

In [ ]:
hz_pp = cnt.ss2tf(Phi_LC, ss.B, ss.C, ss.D, h)
hz_pp

In [ ]:
t = np.linspace(h, h * 100, 100)
r = np.concatenate((np.zeros(int(len(t) / 2)), np.ones(int(len(t) / 2))), axis=None)

In [ ]:
t_hz_pp, y_hz_pp = cnt.forced_response(hz_pp, t, r)

In [ ]:
p_1 = plt.plot(r)
p_2 = plt.plot(y_hz_pp)
p_3 = plt.plot(y_hz_open)
plt.ylabel('u(t)')
plt.title('Respuesta al Escalon')
plt.legend((p_1[0], p_2[0], p_3[0]), ('Señal de entrada', 'Salida controlada', 'Respuesta natural del sistema'));

Ahora probamos obteneniendo K0

In [ ]:
K = cnt.place(ss.A, ss.B, poles)

Phi_LC = ss.A - ss.B * K

In [ ]:
temp1 = np.linalg.inv(np.eye(2) - Phi_LC)
temp2 = np.matmul(ss.C, temp1)
g = np.matmul(temp2, ss.B)
K0 = 1 / g

Gamma_LC = ss.B * K0

In [ ]:
hz_pp = cnt.ss2tf(Phi_LC, Gamma_LC, ss.C, ss.D, h)
hz_pp

**4)** Comparar la respuesta al escalón del punto 2 con la respuesta obtenida aplicando el control por Pole Placement anterior.

In [ ]:
t_hz_pp, y_hz_pp = cnt.forced_response(hz_pp, t, r)

In [ ]:
p_1 = plt.plot(r)
p_2 = plt.plot(y_hz_pp)
p_3 = plt.plot(y_hz_open)
plt.ylabel('u(t)')
plt.title('Respuesta al Escalon')
plt.legend((p_1[0], p_2[0], p_3[0]), ('Señal de entrada', 'Salida controlada', 'Respuesta natural del sistema'));